In [1]:
from dataclasses import dataclass, field
import requests
import datetime as dt

@dataclass
class StarWarsFilms:     
    
    # This __post_init__ method is checking the input data to see if it is valid.
    # (Might need to be a __init__ instead, but then I'll have to change the class structure)
    def __post_init__(self):
        if self.episode_id is None:
            raise Exception("An Episode ID is required!!")
        
        # This adjusts the character input to be complete character objects instead of links
        if self.characters is not None:
            updated_characters = []
            for character in self.characters:
                char = self.get_characters(character)
                print(char['name'])
                attrs = self.get_character_attributes(char)
                updated_characters.append(FilmCharacters(*attrs))
            self.characters = updated_characters
    
    title: str = None
    episode_id: int = None
    opening_crawl: str = None
    director: str = None
    producer: str = None
    release_date: str = None
    characters: list = None
    plot: str = None
    rotten_tomatoes: int = None
    box_office_gross: int = None #int = field(metadata={"units":"U.S. Dollars"}) # Can't seem to use 
    
    # This function is calling the api for the character information
    def get_characters(self, character):
        chars = requests.get(character).json()
        return chars
    
    def get_character_attributes(self,character):
        goal_list = [
            'name',
            'height',
            'mass',
            'hair_color',
            'eye_color',
            'birth_year',
            'gender'
        ]
        
        attr_list = []
        for attr in goal_list:
            if attr in character:
                attr_list.append(character[attr])
            else:
                attr_list.append(None)
        return attr_list
    
@dataclass
class FilmCharacters:
    name: str
    height: float
    mass: float = field(metadata={"units":"kilograms"})
    hair_color: str
    eye_color: str
    birth_year: str
    gender: str

class StarWarsLibrary:
    """ Star Wars Library Class """
    def __init__(self):
        self.star_path = 'https://swapi.dev/api/films'
        self.omdb_path = 'http://www.omdbapi.com/?i=tt3896198&apikey=3ef42ac3'
        self.last_updated = dt.datetime.now()
        self.__star_data = None
        self.__omdb_data = None
        
    def access_star_wars_api(self):
        self.__star_data = requests.get(self.star_path).json()['results']
        
    def access_omdb_api(self):
        self.__ombd_data = requests.get(self.omdb_path).json()
        
    def parse_omdb_data(self):
        target_data = {
            
        }
        example = {"Title":"Guardians of the Galaxy Vol. 2","Year":"2017","Rated":"PG-13","Released":"05 May 2017","Runtime":"136 min","Genre":"Action, Adventure, Comedy","Director":"James Gunn","Writer":"James Gunn, Dan Abnett, Andy Lanning","Actors":"Chris Pratt, Zoe Saldana, Dave Bautista","Plot":"The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father the ambitious celestial being Ego.","Language":"English","Country":"United States","Awards":"Nominated for 1 Oscar. 15 wins & 59 nominations total","Poster":"https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg","Ratings":[{"Source":"Internet Movie Database","Value":"7.6/10"},{"Source":"Rotten Tomatoes","Value":"85%"},{"Source":"Metacritic","Value":"67/100"}],"Metascore":"67","imdbRating":"7.6","imdbVotes":"641,226","imdbID":"tt3896198","Type":"movie","DVD":"22 Aug 2017","BoxOffice":"$389,813,101","Production":"N/A","Website":"N/A","Response":"True"}
        has_rating = False
        for rating in example['Ratings']:
            if rating['Source'] == 'Rotten Tomatoes':
                has_rating = True
                rotten_rating = rating['Value']
                print(rating)
                print(rotten_rating)
        return rating
        
    def update_library(self):
        print('Checking for new movies.')
        self.access_api()
        return self.data
        #for item in self.data:
    
    def functions():
        films = []
        for i,film in enumerate(data):
            print(f'Parsing {film['name']}')
            film_info = {
                'title': None, 
                'episode_id': None, 
                'opening_crawl': None, 
                'director': None, 
                'producer': None, 
                'release_date': None, 
                'characters': None, 
                'plot': None, 
                'rotten_tomatoes': None, 
                'box_office_gross': None    
                }

            for field in interest_data.keys():
                if field in film:
                    film_info[field] = film[field]
            film_obj = StarWarsFilms(*film_info.values())
            films.append(film_obj)
            print("######################")
            break
            

people = path + "people"

build = StarWarsLibrary(films)
data = build.update_library()

In [1]:
char_example = """{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}"""
print(char_example)

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}


In [2]:

class_fields = [
    'title', 
    'episode_id', 
    'opening_crawl', 
    'director', 
    'producer', 
    'release_date', 
    'characters', 
    'plot', 
    'rotten_tomatoes',
    'box_office_gross',]
interest_data = [_ for _ in data[0].keys() if _ in class_fields]


In [4]:

for film in films:
    print(film)

Luke Skywalker
C-3PO
R2-D2
Darth Vader
Leia Organa
Owen Lars
Beru Whitesun lars
R5-D4
Biggs Darklighter
Obi-Wan Kenobi
Wilhuff Tarkin
Chewbacca
Han Solo
Greedo
Jabba Desilijic Tiure
Wedge Antilles
Jek Tono Porkins
Raymus Antilles
######################
StarWarsFilms(title='A New Hope', episode_id=4, opening_crawl="It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....", director='George Lucas', producer='Gary Kurtz, Rick McCallum', release_date='1977-05-25', characters=[FilmCharacters(na

In [10]:
for char in films[0].characters:
    print(char.name)

Luke Skywalker
C-3PO
R2-D2
Darth Vader
Leia Organa
Owen Lars
Beru Whitesun lars
R5-D4
Biggs Darklighter
Obi-Wan Kenobi
Wilhuff Tarkin
Chewbacca
Han Solo
Greedo
Jabba Desilijic Tiure
Wedge Antilles
Jek Tono Porkins
Raymus Antilles


In [6]:
films

[]

In [ ]:
url = 'http://www.omdbapi.com/?i=tt3896198&apikey=3ef42ac3'

In [28]:
example = {"Title":"Guardians of the Galaxy Vol. 2","Year":"2017","Rated":"PG-13","Released":"05 May 2017","Runtime":"136 min","Genre":"Action, Adventure, Comedy","Director":"James Gunn","Writer":"James Gunn, Dan Abnett, Andy Lanning","Actors":"Chris Pratt, Zoe Saldana, Dave Bautista","Plot":"The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father the ambitious celestial being Ego.","Language":"English","Country":"United States","Awards":"Nominated for 1 Oscar. 15 wins & 59 nominations total","Poster":"https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg","Ratings":[{"Source":"Internet Movie Database","Value":"7.6/10"},{"Source":"Rotten Tomatoes","Value":"85%"},{"Source":"Metacritic","Value":"67/100"}],"Metascore":"67","imdbRating":"7.6","imdbVotes":"641,226","imdbID":"tt3896198","Type":"movie","DVD":"22 Aug 2017","BoxOffice":"$389,813,101","Production":"N/A","Website":"N/A","Response":"True"}
has_rating = False
for rating in example['Ratings']:
    if rating['Source'] == 'Rotten Tomatoes':
        has_rating = True
        rotten_rating = rating['Value']
        print(rating)
        print(rotten_rating)

{'Source': 'Rotten Tomatoes', 'Value': '85%'}
85%


In [79]:
for field in class_fields:
    print(f"'{field}': None, ")

'title': None, 
'episode_id': None, 
'opening_crawl': None, 
'director': None, 
'producer': None, 
'release_date': None, 
'characters': None, 
'plot': None, 
'rotten_tomatoes': None, 
'box_office_gross': None, 


In [69]:
strig  = """title: str
    episode_id: int
    opening_crawl: str
    director: str
    producer: str
    release_date: str
    characters: list
    plot: str
    rotten_tomatoes: int
    box_office_gross: int = field(metadata={"units":"U.S. Dollars"})"""

strig = strig.replace("\r","").replace("\n","").strip().split(":")
strig = [_.replace("str   ","").replace('int   ',"").strip() for _ in strig]
print(strig)

['title', 'episode_id', 'opening_crawl', 'director', 'producer', 'release_date', 'characters', 'list    plot', 'rotten_tomatoes', 'box_office_gross', 'int = field(metadata={"units"', '"U.S. Dollars"})']
